In [1]:
meta_path = '/content/drive/MyDrive/Histopathology/Data/TCGA_KIRC/split_df/train_df_4.csv'
gene_path = '/content/drive/MyDrive/Histopathology/Data/TCGA_KIRC/gene_dfs.pt'
emb_path = '/content/drive/MyDrive/Histopathology/Data/TCGA_KIRC/wsi_embeddings/train_4_2.pt'
save_path = '/content/drive/MyDrive/Histopathology/Data/TCGA_KIRC/processed_data/train/'

# Imports

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import joblib

# Load data

In [4]:
gene_df = torch.load(gene_path)
gene_df

<ipython-input-4-eb541a51dd04>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  gene_df = torch.load(gene_path)


{'Tumor Suppressor Genes': Hugo_Symbol          case_id    APC2     APC  APCDD1 APCDD1L    APCS   ASXL1  \
 0            TCGA-3Z-A93Z-01   0.815 -0.8616 -0.4042 -0.1023 -0.1148 -0.5621   
 1            TCGA-6D-AA2E-01 -0.1376  -0.383 -0.6515  -0.243 -0.1011 -0.9872   
 2            TCGA-A3-3306-01    0.39 -0.7711 -1.1005 -0.2473 -0.1148 -1.2054   
 3            TCGA-A3-3307-01 -0.2787  0.1075  -0.021 -0.2473 -0.1127  -0.751   
 4            TCGA-A3-3308-01 -0.0956  0.4379  2.2967 -0.1205 -0.1148  0.9187   
 ..                       ...     ...     ...     ...     ...     ...     ...   
 505          TCGA-MM-A563-01 -0.0313 -0.9168 -0.4426  -0.218 -0.0869 -0.4564   
 506          TCGA-MM-A564-01  0.4541 -1.0614  1.9283 -0.2306 -0.1148 -0.1561   
 507          TCGA-MM-A84U-01 -0.0542 -1.3949 -0.5626  1.4628 -0.0719 -0.2128   
 508          TCGA-MW-A4EC-01   -0.17 -0.7995 -0.8115 -0.2375 -0.1148 -0.6339   
 509          TCGA-T7-A92I-01  0.1092  2.6542  -0.865 -0.2434 -0.1148 -0.7806   
 


In [5]:
for key, df in gene_df.items():
    gene_df[key] = gene_df[key].dropna(axis=1)
    gene_df[key]['case_id'] = gene_df[key]['case_id'].apply(lambda x: x[:-3])
    print(gene_df[key].shape[1])

145
522
669
681
2366
990


<ipython-input-5-1fe0aa1bd5ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df[key]['case_id'] = gene_df[key]['case_id'].apply(lambda x: x[:-3])
<ipython-input-5-1fe0aa1bd5ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_df[key]['case_id'] = gene_df[key]['case_id'].apply(lambda x: x[:-3])
<ipython-input-5-1fe0aa1bd5ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [6]:
gene_df.keys()

dict_keys(['Tumor Suppressor Genes', 'Oncogenes', 'Protein Kinases', 'Cell Differentiation Markers', 'Transcription Factors', 'Cytokines and Growth Factors'])

In [7]:
has_nan = gene_df['Cytokines and Growth Factors'].isnull().values.any()
has_nan

False

In [8]:
gene_df['Tumor Suppressor Genes']

Hugo_Symbol,case_id,APC2,APC,APCDD1,APCDD1L,APCS,ASXL1,ATM,ATMIN,BLM,...,TSC22D3,TSC22D4,TSC2,VHL,VHLL,WRN,WRNIP1,WT1,XPA,XPC
0,TCGA-3Z-A93Z,0.815,-0.8616,-0.4042,-0.1023,-0.1148,-0.5621,-1.1316,-1.4028,-0.693,...,1.0879,0.207,-0.0171,-0.5392,-0.2616,-0.9342,1.2646,0.2124,1.4834,0.0814
1,TCGA-6D-AA2E,-0.1376,-0.383,-0.6515,-0.243,-0.1011,-0.9872,-0.8601,-0.8684,-1.0127,...,-0.1201,1.5478,-0.5192,1.2264,-0.2616,-0.211,0.9023,-0.5005,1.4556,-0.0973
2,TCGA-A3-3306,0.39,-0.7711,-1.1005,-0.2473,-0.1148,-1.2054,2.3433,-1.3811,-0.5898,...,-0.2169,-0.9034,-0.2093,0.1147,-0.2616,1.0934,-1.3567,-0.3512,-0.6276,-0.4061
3,TCGA-A3-3307,-0.2787,0.1075,-0.021,-0.2473,-0.1127,-0.751,0.6795,0.752,-0.6213,...,0.8839,-1.0701,-0.1338,0.5283,0.6129,0.0463,0.0967,-0.433,0.3212,-0.9139
4,TCGA-A3-3308,-0.0956,0.4379,2.2967,-0.1205,-0.1148,0.9187,0.7144,0.754,-0.415,...,-0.5035,-0.4929,0.6776,0.8944,-0.2616,0.8859,-0.2931,0.0779,-0.6338,-0.9448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,TCGA-MM-A563,-0.0313,-0.9168,-0.4426,-0.218,-0.0869,-0.4564,-0.4477,-0.9006,0.7968,...,-0.122,0.7548,-0.4188,0.0039,-0.2616,-0.3616,0.0278,-0.4862,0.0143,0.5393
506,TCGA-MM-A564,0.4541,-1.0614,1.9283,-0.2306,-0.1148,-0.1561,0.5164,-2.0573,-0.4542,...,-0.8485,4.3998,-0.2248,-0.8404,-0.2616,-0.5831,1.5121,0.0041,1.6626,-0.7805
507,TCGA-MM-A84U,-0.0542,-1.3949,-0.5626,1.4628,-0.0719,-0.2128,-0.8915,-1.4208,-0.42,...,-0.5671,0.5902,-0.1032,-1.0306,-0.2616,-0.837,-0.0046,0.0028,-0.3567,0.1059
508,TCGA-MW-A4EC,-0.17,-0.7995,-0.8115,-0.2375,-0.1148,-0.6339,-0.4869,-0.4162,-0.6874,...,0.2056,-0.3687,-0.3199,-0.1342,-0.2616,-0.0907,3.3834,-0.5005,1.8797,0.2906


In [9]:
df_meta = pd.read_csv(meta_path)
df_meta

,id,age,gender,race,primary_diagnosis,prior_malignancy,grade,censorship,survival_months,time,time_bin
0,TCGA-CW-6097,32,Male,white,"Clear cell adenocarcinoma, NOS",no,4,0,18.772397,0.125854,1
1,TCGA-BP-4352,74,Female,white,"Clear cell adenocarcinoma, NOS",no,4,0,11.309465,0.075821,1
2,TCGA-CJ-4890,72,Male,white,"Clear cell adenocarcinoma, NOS",no,4,1,115.691883,0.775623,4
3,TCGA-B0-4699,74,Male,white,"Clear cell adenocarcinoma, NOS",no,4,0,3.616399,0.024245,1
4,TCGA-B0-5696,69,Male,white,"Clear cell adenocarcinoma, NOS",no,4,1,85.774402,0.575050,3
5,TCGA-B0-5109,69,Male,white,"Clear cell adenocarcinoma, NOS",no,4,0,19.298419,0.129381,1
6,TCGA-BP-4770,73,Female,white,"Clear cell adenocarcinoma, NOS",no,4,0,10.816320,0.072515,1
7,TCGA-CJ-6033,54,Female,white,"Clear cell adenocarcinoma, NOS",no,4,0,7.364303,0.049372,1
8,TCGA-CJ-6027,77,Male,white,"Clear cell adenocarcinoma, NOS",yes,4,0,118.848013,0.796782,4
9,TCGA-CJ-5689,90,Male,white,"Clear cell adenocarcinoma, NOS",yes,4,0,53.259690,0.357064,2


# Process data

In [10]:
emb = torch.load(emb_path)

<ipython-input-10-25e71a933f3d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  emb = torch.load(emb_path)


In [11]:
len(emb)

75

In [13]:
#Z-score normalize all dataframes in gene_df separately, except for the first column of each dataframe.
normalized_gene_df = {}
mmscaler = []
for key, df in gene_df.items():
    # Exclude the first column and perform z-score normalization
    scaler = MinMaxScaler()
    df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:].values)
    #df.loc[:, df.columns[1:]] = scaler.fit_transform(df.iloc[:, 1:])
    normalized_gene_df[key] = df

    mmscaler.append(scaler)

joblib.dump(mmscaler, '/content/drive/MyDrive/Histopathology/Data/TCGA_KIRC/org_minmax_scaler.pkl')
# Normalize the 'survival_months' column in df_meta.
#survival_scaler = MinMaxScaler()
#df_meta['survival_months_normalized'] = survival_scaler.fit_transform(df_meta[['survival_months']])

['/content/drive/MyDrive/Histopathology/Data/TCGA_KIRC/org_minmax_scaler.pkl']

In [168]:
# Construct the dictionary for each element in emb.
constructed_dicts = []
for emb_elem in emb:
    emb_dict = {}
    # Add TCGA ID and slide ID
    emb_dict['TCGA ID'] = emb_elem['TCGA ID']
    emb_dict['slide_id'] = emb_elem['slide_id']

    # Add WSI embedding
    emb_dict['WSI_embedding'] = emb_elem['embedding']

    # Add omic data
    flag = False
    for key, df in normalized_gene_df.items():
        omic_data = df.loc[df['case_id'] == emb_elem['TCGA ID'][:-2]].iloc[:, 1:].values
        if(len(omic_data) == 0):
            flag = True
        emb_dict[f'omic_data_{key}'] = torch.tensor(omic_data.astype(np.float32))
    if(flag):
        print(emb_elem['TCGA ID'][:-2])
        continue

    # Add metadata
    meta_data = df_meta.loc[df_meta['id'] == emb_elem['TCGA ID'][:-2]].iloc[0]
    emb_dict['meta_data'] = {
        'level': emb_elem['level'],
        'grade': emb_elem['grade'],
        'diagnosis': meta_data['primary_diagnosis'],
        'is_female': 1.0 if meta_data['gender'] == 'Female' else 0.0,
        'age': meta_data['age'],
        'race': meta_data['race'],
        'survival_months': meta_data['survival_months'],
        'time': meta_data['time'],
        'time_bin': meta_data['time_bin']
        }

    # Add label
    grade = emb_elem['grade']
    if grade == 1:
        emb_dict['label'] = [1.0, 0.0, 0.0, 0.0]
    elif grade == 2:
        emb_dict['label'] = [0.0, 1.0, 0.0, 0.0]
    elif grade == 3:
        emb_dict['label'] = [0.0, 0.0, 1.0, 0.0]
    else:
        emb_dict['label'] = [0.0, 0.0, 0.0, 1.0]

    # Add risk and censorship
    time_bin = meta_data['time_bin']
    time_label = [0.0, 0.0, 0.0, 0.0]
    time_label[time_bin-1] = 1.0
    emb_dict['time_label'] = time_label
    emb_dict['censorship'] = meta_data['censorship']

    # Save emb_dict
    torch.save(emb_dict, f'{save_path}{emb_elem["slide_id"]}.pt')

    constructed_dicts.append(emb_dict)

# constructed_dicts contains the constructed dictionaries for each element in emb.


In [169]:
len(constructed_dicts)

75

In [170]:
constructed_dicts[0]

{'TCGA ID': 'TCGA-BP-4992_1',
 'slide_id': 'TCGA-BP-4992-01Z-00-DX1.3f676d40-00de-4eb4-971d-6b93ed165cf5.svs_1',
 'WSI_embedding': tensor([[ 0.4423, -0.6556,  0.3245,  ..., -0.3939,  1.2857,  2.0457],
         [-0.5472, -0.7072,  1.5793,  ..., -0.1630,  0.1347,  1.4977],
         [-0.2064, -0.8318,  1.3889,  ..., -1.1322,  1.6972,  1.3668],
         ...,
         [-0.6994, -0.8824,  1.1569,  ...,  0.0254, -1.7501,  0.5182],
         [-0.4491,  0.2990,  1.6471,  ..., -0.6780, -0.2967,  2.8338],
         [-0.6017,  0.2498,  0.8277,  ..., -1.3334, -0.5930,  1.4238]]),
 'omic_data_Tumor Suppressor Genes': tensor([[0.0943, 0.2254, 0.0834, 0.0061, 0.0000, 1.0000, 0.4603, 0.0736, 0.1368,
          0.2355, 0.1094, 0.0731, 0.1014, 0.0427, 0.0706, 0.4958, 0.0041, 0.0613,
          0.0080, 0.0333, 0.0156, 0.0320, 0.0012, 0.0780, 0.0021, 0.0056, 0.0077,
          0.0219, 0.0443, 0.2316, 0.1837, 0.1213, 0.2547, 0.3869, 0.2022, 0.5148,
          0.3551, 0.1264, 0.7928, 0.2560, 0.4945, 0.0404, 0.0557

In [171]:
dt = torch.load(save_path+constructed_dicts[0]['slide_id']+'.pt')
dt

<ipython-input-171-6996db91fc4d>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dt = torch.load(save_path+constructed_dicts[0]['slide_id']+'.pt')


{'TCGA ID': 'TCGA-BP-4992_1',
 'slide_id': 'TCGA-BP-4992-01Z-00-DX1.3f676d40-00de-4eb4-971d-6b93ed165cf5.svs_1',
 'WSI_embedding': tensor([[ 0.4423, -0.6556,  0.3245,  ..., -0.3939,  1.2857,  2.0457],
         [-0.5472, -0.7072,  1.5793,  ..., -0.1630,  0.1347,  1.4977],
         [-0.2064, -0.8318,  1.3889,  ..., -1.1322,  1.6972,  1.3668],
         ...,
         [-0.6994, -0.8824,  1.1569,  ...,  0.0254, -1.7501,  0.5182],
         [-0.4491,  0.2990,  1.6471,  ..., -0.6780, -0.2967,  2.8338],
         [-0.6017,  0.2498,  0.8277,  ..., -1.3334, -0.5930,  1.4238]]),
 'omic_data_Tumor Suppressor Genes': tensor([[0.0943, 0.2254, 0.0834, 0.0061, 0.0000, 1.0000, 0.4603, 0.0736, 0.1368,
          0.2355, 0.1094, 0.0731, 0.1014, 0.0427, 0.0706, 0.4958, 0.0041, 0.0613,
          0.0080, 0.0333, 0.0156, 0.0320, 0.0012, 0.0780, 0.0021, 0.0056, 0.0077,
          0.0219, 0.0443, 0.2316, 0.1837, 0.1213, 0.2547, 0.3869, 0.2022, 0.5148,
          0.3551, 0.1264, 0.7928, 0.2560, 0.4945, 0.0404, 0.0557